<a href="https://colab.research.google.com/github/Saksham9804/Projects/blob/main/Data_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***SETUP***

Installing required library for PDF generation


In [ ]:
!pip install reportlab

Importing necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import kagglehub
from kagglehub import KaggleDatasetAdapter
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

# Machine learning related imports
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier


Loading the dataset

In [ ]:
# Downloading latest version
path = kagglehub.dataset_download("sakshamtiwari98/market-analysis")

print("Path to dataset files:", path)

df = pd.read_csv('/kaggle/input/market-analysis/Dataset.csv')
print(df.head())

Path to dataset files: /kaggle/input/market-analysis
       Sort   Invoice ID Branch       City Customer type  Gender  \
0  0.721157  811-35-1094    NaN   Mandalay        Member    Male   
1  0.150600  546-80-2899      A     Yangon        Member    Male   
2  0.946204  449-27-2918      B   Mandalay        Member  Female   
3  0.423019  503-07-0930      C  Naypyitaw        Member    Male   
4  0.430853  592-46-1692      C  Naypyitaw        Member  Female   

             Product line  Unit price Quantity   Tax 5%     Total        Date  \
0  Electronic accessories       50.45      Six  15.1350  317.8350  02-06-2019   
1      Home and lifestyle       37.69        2   3.7690   79.1490   2/20/2019   
2       Sports and travel         NaN        1   1.9560   41.0760   3/26/2019   
3       Sports and travel       58.39        7  20.4365  429.1665   2/23/2019   
4      Food and beverages       36.77        7  12.8695  270.2595  01-11-2019   

    Time      Payment    cogs  gross margin percent

***Data Cleaning and Processing***

Dictionary to convert number words to numeric values

In [ ]:
number_words = {
    'zero': 0, 'one': 1, 'two': 2, 'three': 3, 'four': 4,
    'five': 5, 'six': 6, 'seven': 7, 'eight': 8, 'nine': 9,
    'ten': 10, 'eleven': 11, 'twelve': 12
}

Making the function to handle word-to-number conversion

In [ ]:
def word_to_num(x):
    if isinstance(x, str):
        return number_words.get(x.lower(), x)
    return x

 Converting the 'Date'and 'Time'  column to datetime format

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

df['Time'] = pd.to_datetime(df['Time'], format='%H:%M', errors='coerce').dt.time

Converting the numeric columns (handling both words and numbers)

In [ ]:
for col in ['Quantity', 'Unit price', 'Rating']:
    df[col] = pd.to_numeric(df[col].apply(word_to_num), errors='coerce')

Now I'm droping the rows where essential fields are missing

In [ ]:
df.dropna(subset=['Date', 'Time', 'Quantity', 'Unit price', 'Rating'], inplace=True)

Also creating additional derived feature: Day of the Week

In [ ]:
df['DayOfWeek'] = df['Date'].dt.day_name()
df['Hour'] = df['Time'].apply(lambda x: x.hour)

Making the Function to derive "Time of Day" category from time

In [ ]:
def get_time_of_day(t):
    if pd.isnull(t):
        return 'Unknown'
    t = datetime.datetime.strptime(str(t), "%H:%M:%S").time()
    if datetime.time(5, 0) <= t < datetime.time(12, 0):
        return 'Morning'
    elif datetime.time(12, 0) <= t < datetime.time(17, 0):
        return 'Afternoon'
    elif datetime.time(17, 0) <= t < datetime.time(21, 0):
        return 'Evening'
    else:
        return 'Night'

Applying time-of-day categorization and standardizing customer format

In [ ]:
df['TimeOfDay'] = df['Time'].apply(get_time_of_day)

df['Customer type'] = df['Customer type'].str.strip().str.title()

***PREVIEWING DATA***

In [ ]:
pd.set_option('display.max_columns', None)
file_path = '/content/Dataset.csv'
df = pd.read_csv(file_path)

# Inspect first few rows and dataset info
print("\nDataset Head:")
print(df.head())
df.info()
print("\nSummary Statistics:")
print(df.describe())

***EXPLORATORY DATA ANALYSIS (EDA)***

In [ ]:
sns.set(style="whitegrid")

a. Getting Revenue by branch

In [ ]:
branch_revenue = df.groupby('Branch')['Total'].sum().sort_values(ascending=False)
print("\nTotal Revenue by Branch:\n", branch_revenue)
branch_revenue.plot(kind='bar', title='Total Revenue by Branch')
plt.show()

b. Sales by product line

In [ ]:
product_sales = df.groupby('Product line')['Total'].sum().sort_values(ascending=False)
print("\nTotal Sales by Product Line:\n", product_sales)
sns.barplot(x=product_sales.values, y=product_sales.index)
plt.title('Sales by Product Line')
plt.show()

c. Taking Average transaction per city

In [ ]:
print("\nAverage Transaction Value per City:")
print(df.groupby('City')['Total'].mean())


d. Total Spend by Customer Type

In [ ]:
print("\nAverage Spend by Customer Type:")
print(df.groupby('Customer type')['Total'].mean())

e. Spendings according to the Gender

In [ ]:
sns.boxplot(x='Gender', y='Total', data=df)
plt.title('Spend by Gender')
plt.show()

f. Preferred Payment Method

In [ ]:
print("\nPreferred Payment Methods:")
print(df['Payment'].value_counts())
sns.countplot(data=df, x='Payment')
plt.title('Preferred Payment Methods')
plt.show()

g. Customer type distribution by city

In [ ]:
sns.countplot(data=df, x='City', hue='Customer type')
plt.title('Customer Type by City')
plt.show()

h. Peak shopping hours

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M', errors='coerce').dt.time


sns.histplot(df['Time'].apply(lambda x: x.hour), bins=24)
plt.title('Transactions by Hour')
plt.xlabel('Hour')
plt.show()

i. Highest average rating by product line

In [ ]:
print("\nAverage Rating by Product Line:")
print(df.groupby('Product line')['Rating'].mean().sort_values(ascending=False))

j. Correlation matrix

In [ ]:
# Re-apply data cleaning for 'Quantity' column
df['Quantity'] = df['Quantity'].apply(word_to_num)
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')
df.dropna(subset=['Quantity', 'Unit price', 'Rating'], inplace=True) # Drop rows with missing values after conversion

sns.heatmap(df[['Unit price', 'Quantity', 'Rating']].corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

k. Price vs Rating scatter plot

In [ ]:
sns.scatterplot(x='Unit price', y='Rating', data=df)
plt.title('Rating vs Unit Price')
plt.show()

l. Gross income by branch

In [ ]:
print("\nGross Income by Branch:")
print(df.groupby('Branch')['gross income'].sum().sort_values(ascending=False))

m. Profit per unit calculation

In [ ]:
profit_per_unit = df.groupby('Product line')['gross income'].sum() / df.groupby('Product line')['Quantity'].sum()
profit_per_unit = profit_per_unit.sort_values(ascending=False)
print("\nProfit per Unit Sold:")
print(profit_per_unit)

n. Transactions per hour

In [ ]:
print("\nTransactions Per Hour:")
print(df['Time'].apply(lambda x: x.hour).value_counts().sort_index())

***MODELLING***

Regression task: Predict Rating

In [ ]:
X_reg = df[['Unit price', 'Quantity', 'gross income']]
y_reg = df['Rating']
X_train, X_test, y_train, y_test = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

model_reg = LinearRegression()
model_reg.fit(X_train, y_train)
y_pred = model_reg.predict(X_test)

print("\nRegression Performance:")
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("R² Score:", r2_score(y_test, y_pred))

Classification: Predicting Customer Type

In [ ]:
df['TimeOfDay'] = df['Time'].apply(get_time_of_day)

df_model = df.copy()

df_model['DayOfWeek'] = df_model['Date'].dt.day_name()

Maping Customer type to numbers

In [ ]:
df_model['Customer type'] = df_model['Customer type'].str.strip().str.title()
df_model['Customer type'] = df_model['Customer type'].map({'Member': 1, 'Normal': 0})
df_model.dropna(subset=['Customer type'], inplace=True)

Encoding categorical variables

In [ ]:

le = LabelEncoder()
for col in ['Gender', 'Product line', 'Payment', 'City', 'Branch', 'TimeOfDay', 'DayOfWeek']:
    df_model[col] = le.fit_transform(df_model[col])

Selecting features for classification

In [ ]:
X_cls = df_model.drop(['Customer type', 'Invoice ID', 'Date', 'Time', 'Rating'], axis=1)
y_cls = df_model['Customer type']
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_cls, y_cls, test_size=0.2, random_state=42)

Training Random Forest Classifier

In [ ]:
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_c, y_train_c)
y_pred_c = clf.predict(X_test_c)

print("\nClassification Performance:")
print("Accuracy:", accuracy_score(y_test_c, y_pred_c))
print(classification_report(y_test_c, y_pred_c))

***SUMMARIZED ANSWERS FOR EDA (Part 3)***

In [ ]:
# Create the 'Hour' column from the processed 'Time' column
df['Hour'] = df['Time'].apply(lambda x: x.hour)

answers = [
    "a. The branch with the highest revenue is - " + df.groupby('Branch')['Total'].sum().idxmax(),
    "b. Most contributing product line is -  " + df.groupby('Product line')['Total'].sum().idxmax(),
    "c. The city with highest average transaction is - " + df.groupby('City')['Total'].mean().idxmax(),
    "d. Higher spendings is by - " + df.groupby('Customer type')['Total'].mean().idxmax(),
    "e. The  highest spending gender is - " + df.groupby('Gender')['Total'].mean().idxmax(),
    "f. The most used payment method is -  " + df['Payment'].value_counts().idxmax(),
    "g. The most common type overall is - " + df.groupby('City')['Customer type'].agg(lambda x: x.mode()[0]).mode()[0],
    "h. The peak hour is - " + str(df['Hour'].value_counts().idxmax()) + ":00",
    "i. The highest avg rating line is - " + df.groupby('Product line')['Rating'].mean().idxmax(),
    "j. The correlation between price & quantity is - " + df[['Unit price', 'Quantity', 'Rating']].corr().loc['Unit price', 'Quantity'].round(2).astype(str),
    "k. The effect of price on rating is - " + ("Higher" if df[['Unit price', 'Rating']].corr().iloc[0,1] > 0 else "Lower"),
    "l. The highest gross income branch is -  " + df.groupby('Branch')['gross income'].sum().idxmax(),
    "m. The most profitable per unit is -  " + (df.groupby('Product line')['gross income'].sum() / df.groupby('Product line')['Quantity'].sum()).idxmax(),
    "n. The peak hour again for off-peak/peak - " + str(df['Hour'].value_counts().idxmax()) + ":00"
]

Finally, saving the answers as PDF

In [ ]:
pdf_path = "Part 3 Answers.pdf"
c = canvas.Canvas(pdf_path, pagesize=letter)
width, height = letter
text = c.beginText(40, height - 40)
text.setFont("Helvetica", 11)

text.textLine("PART 3  Answers")
text.textLine("-" * 40)

# Writing each answer lines to PDF
for ans in answers:
    text.textLine(ans)

# Saving the PDF file
c.drawText(text)
c.save()

print(" PDF saved as:", pdf_path)